# 1.ADIM : 

# Makine öğrenmesi ve derin öğrenme için gerekli kütüphaneler ile dosya yolu işlemleri için kullanılan yardımcı modüllerin yüklenmesi

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

from pathlib import Path
import os


# 2. ADIM : 

# Görüntü dosyalarının yolları ve etiketleri belirlenip bir DataFrame'e dönüştürülmesi, GT görüntülerinin çıkarılması ve veri setinin karıştırılarak yeniden düzenlenmesi

In [ ]:
image_dir = Path('/kaggle/input/a-large-scale-fish-dataset')

filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df = pd.concat([filepaths, labels], axis=1)
image_df = image_df[image_df['Label'].apply(lambda x: x[-2:] != 'GT')]

image_df = image_df.sample(frac=1).reset_index(drop=True)

print(image_df.head())


# 3. ADIM : 

# İlk 15 görüntü, 224x224 boyutunda yüklenip normalize edilerek etiketleriyle beraber görselleştirilmesi

In [ ]:
from tensorflow.keras.utils import load_img, img_to_array

fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(15, 7), subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    img_path = image_df.Filepath[i]
    
    try:
        img = load_img(img_path, target_size=(224, 224))  # Görüntü boyutunu 224x224 olarak ayarla
        img_array = img_to_array(img) / 255.0  # Normalizasyon
        
        ax.imshow(img_array)
        ax.set_title(image_df.Label[i])
    except Exception as e:
        print(f"Görüntü yüklenirken hata oluştu: {e}")

plt.tight_layout()
plt.show()


# 4. ADIM : 

# Az örneği olan sınıfların çıkarılması ve verilerin her sınıfın eşit dağılımını sağlamak için stratify ile eğitim ve test setlerine ayrılması


In [ ]:
class_counts = image_df['Label'].value_counts()

few_samples_classes = class_counts[class_counts < 2].index
print("Az Örnekli Sınıflar:\n", few_samples_classes)

filtered_image_df = image_df[~image_df['Label'].isin(few_samples_classes)]
print(f"Filtrelenmiş veri setinde {len(filtered_image_df)} görüntü kaldı.")

train_df, test_df = train_test_split(
    filtered_image_df, 
    train_size=0.9, 
    shuffle=True, 
    random_state=1, 
    stratify=filtered_image_df['Label']  
)

train_class_distribution = train_df['Label'].value_counts()
test_class_distribution = test_df['Label'].value_counts()
print("Eğitim Seti Sınıf Dağılımı:\n", train_class_distribution)
print("Test Seti Sınıf Dağılımı:\n", test_class_distribution)


# 5.ADIM : 

# Görüntülerin yüklenmesi, normalleştirilmesi ve veri artırma teknikleriyle eğitim, doğrulama ve test setlerinin hazırlanması


In [ ]:
# ImageDataGenerator ile görüntüleri yükleyip normalleştirelim
train_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,  # Daha fazla döndürme
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.4,  # Yakınlaştırma artırıldı
    brightness_range=[0.6, 1.4],  # Parlaklık genişletildi
    channel_shift_range=0.3,  # Renk kanalı kayması
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)


test_generator = ImageDataGenerator(rescale=1./255)

# Eğitim ve doğrulama setlerini hazırlayalım
# Batch size 64 olarak güncelleniyor
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=256,  # Daha büyük batch size ile denge sağlıyoruz
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=256,  # Validation için de aynı
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=256,  # Test seti için de batch size aynı
    shuffle=False
)
# Eğitim, doğrulama ve test setlerinin boyutlarını yazdıralım
print(f"Toplam {train_images.samples} eğitim görüntüsü, {val_images.samples} doğrulama görüntüsü, {test_images.samples} test görüntüsü yüklendi.")


# 6. ADIM : 

# ANN modelinin oluşturulması

In [ ]:
from tensorflow.keras.layers import BatchNormalization, Input

model = Sequential()

model.add(Flatten(input_shape=(224, 224, 3))) 

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))  

model.summary()



# 7.ADIM: 

# Modelin derlenmesi


In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.0005),  
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


# 8. ADIM : 

# Modelin eğitilmesi

In [ ]:
# Erken durdurma tanımlayalım
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Öğrenme oranını düşürme tanımlayalım
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=7,
    min_lr=1e-6
)

# Modeli eğitelim
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=20,  # Epoch sayısını artırabiliriz
    callbacks=[early_stopping, reduce_lr],  # Hem erken durdurma hem de öğrenme oranı azaltma kullanılıyor
    verbose=1
)


# 9.ADIM : 

# Modelin eğitim ve doğrulama süreçlerindeki doğruluk ve kayıpları analiz etmek için epoch'lar boyunca elde edilen verilerin grafiklerle gösteriminin yapılması.

In [ ]:
# Eğitim ve doğrulama doğruluk ve kayıpları grafiklerini çizdirelim
plt.figure(figsize=(12, 5))

# Doğruluk grafiği
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Model Doğruluğu')
plt.xlabel('Epochs')
plt.ylabel('Doğruluk')
plt.legend()

# Kayıp grafiği
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Model Kayıp')
plt.xlabel('Epochs')
plt.ylabel('Kayıp')
plt.legend()

plt.tight_layout()
plt.show()


# 10 : 

# Modelin test seti üzerinde yaptığı tahminlerin doğruluğunu ölçmek için confusion matrix ve classification report ile performans değerlendirmesinin yapılması.

In [ ]:
# Test setinde tahminler yapalım
pred = model.predict(test_images)
pred_classes = np.argmax(pred, axis=1)

# Gerçek etiketleri alalım
true_classes = test_images.classes
class_labels = list(test_images.class_indices.keys())

# Confusion Matrix ve Classification Report
conf_matrix = confusion_matrix(true_classes, pred_classes)

plt.figure(figsize=(10, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.xlabel('Tahmin Edilen Sınıf')
plt.ylabel('Gerçek Sınıf')
plt.show()

# Sınıflandırma raporu
print(classification_report(true_classes, pred_classes, target_names=class_labels))
